In [ ]:
# ref： https://cooltools.readthedocs.io/en/latest/notebooks/viz.html


In [ ]:
import os 
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline/")
print(os.getcwd())

# read mcool files

In [ ]:
! find . -name *rep12.merged.1k.zoomify.mcool

In [ ]:
nora_wt_rep12_mcool = "./Nora_WT/Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_rep12_mcool = "./Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
wn_bpk25_rep12_mcool = "./WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.1k.zoomify.mcool"


In [ ]:
import cooler
# to print which resolutions are stored in the mcool, use list_coolers
cooler.fileops.list_coolers(nora_wt_rep12_mcool)


In [ ]:
### to load a cooler with a specific resolution use the following syntax:
nora_wt_rep12_mcool_r20k = cooler.Cooler(f'{nora_wt_rep12_mcool}::resolutions/20000')
nora_iaa_rep12_mcool_r20k = cooler.Cooler(f'{nora_iaa_rep12_mcool}::resolutions/20000')
wn_bpk25_rep12_mcool_r20k = cooler.Cooler(f'{wn_bpk25_rep12_mcool}::resolutions/20000')


# read TAD bed files

In [ ]:
! find . -name *.rep12.merged.mcool_r20k.w5r.TADs.bed

In [ ]:
nora_wt_rep12_r20k_tads_file = './Nora_WT/Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.w5r.TADs.bed'
nora_iaa_rep12_r20k_tads_file = './Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.w5r.TADs.bed'
wn_bpk25_rep12_r20k_tads_file = './WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.mcool_r20k.w5r.TADs.bed'


In [ ]:
import pandas as pd
nora_wt_rep12_r20k_tads_df = pd.read_csv(nora_wt_rep12_r20k_tads_file, sep="\t", header=None)
nora_iaa_rep12_r20k_tads_df = pd.read_csv(nora_iaa_rep12_r20k_tads_file, sep="\t", header=None)
wn_bpk25_rep12_r20k_tads_df = pd.read_csv(wn_bpk25_rep12_r20k_tads_file, sep="\t", header=None)
nora_wt_rep12_r20k_tads_df.columns = ["chrom", "start", "end"]
nora_iaa_rep12_r20k_tads_df.columns = ["chrom", "start", "end"]
wn_bpk25_rep12_r20k_tads_df.columns = ["chrom", "start", "end"]
print(nora_wt_rep12_r20k_tads_df.shape, nora_iaa_rep12_r20k_tads_df.shape, wn_bpk25_rep12_r20k_tads_df.shape, sep="\t")
nora_wt_rep12_r20k_tads_df.head()

# read loop info files

In [ ]:
! find . -name *rep12.merged.mcool_r20k.dots_info.tsv

In [ ]:
nora_wt_rep12_r20k_loops_file = './Nora_WT/Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.dots_info.tsv'
nora_iaa_rep12_r20k_loops_file = './Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.mcool_r20k.dots_info.tsv'
wn_bpk25_rep12_r20k_loops_file = './WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.mcool_r20k.dots_info.tsv'


In [ ]:
import pandas as pd
nora_wt_rep12_r20k_loops_df = pd.read_csv(nora_wt_rep12_r20k_loops_file, sep="\t", header=0)
nora_iaa_rep12_r20k_loops_df = pd.read_csv(nora_iaa_rep12_r20k_loops_file, sep="\t", header=0)
wn_bpk25_rep12_r20k_loops_df = pd.read_csv(wn_bpk25_rep12_r20k_loops_file, sep="\t", header=0)
print(nora_wt_rep12_r20k_loops_df.shape, nora_iaa_rep12_r20k_loops_df.shape, wn_bpk25_rep12_r20k_loops_df.shape, sep="\t")
nora_wt_rep12_r20k_loops_df.head()


# functions for adding tads and loops

In [ ]:
import bioframe
# Visualization imports:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter
# helper functions for plotting
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    """format ticks with genomic coordinates as human readable"""
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)


# create a functions that would return a series of rectangles around called tads
# in a specific region, and exposing importnat plotting parameters
def rectangles_around_tads(tads_df, plot_region, loc="none", lw=1, ec="lime", fc="none"):
    """
    yield a series of rectangles around called tads in a given region
    """
    # select tads from the region:
    df_reg = bioframe.select(tads_df, plot_region, cols=("chrom", "start", "end"))
    Polygon_kwargs = dict(lw=lw, edgecolor=ec, facecolor=fc)
    # draw rectangular "boxes" around pixels called as tads in the "region":
    for tad_s, tad_e in df_reg[["start", "end"]].itertuples(index=False):
        if loc == "upper":
            # 注意：多边形顶点顺序需要按照逆时针或顺时针排列
            yield patches.Polygon([(tad_s, tad_s), (tad_e, tad_e), (tad_e, tad_s)], closed=True, **Polygon_kwargs)
        elif loc == "lower":
            yield patches.Polygon([(tad_s, tad_s), (tad_s, tad_e), (tad_e, tad_e)], closed=True, **Polygon_kwargs)
        else:
            yield patches.Polygon([(tad_s, tad_s), (tad_s, tad_e), (tad_e, tad_e), (tad_e, tad_s)], closed=True, **Polygon_kwargs)


# create a functions that would return a series of rectangles around called dots
# in a specific region, and exposing importnat plotting parameters
def rectangles_around_dots(dots_df, resolution, region, loc="upper", lw=1, ec="cyan", fc="none"):
    """
    yield a series of rectangles around called dots in a given region
    """
    # select dots from the region:
    df_reg = bioframe.select(
        bioframe.select(dots_df, region, cols=("chrom1","start1","end1")),
        region,
        cols=("chrom2","start2","end2"),
    )
    rectangle_kwargs = dict(lw=lw, ec=ec, fc=fc)
    # draw rectangular "boxes" around pixels called as dots in the "region":
    for s1, s2, e1, e2 in df_reg[["start1", "start2", "end1", "end2"]].itertuples(index=False):
        width1 = e1 - s1
        width2 = e2 - s2
        if loc == "upper":
            yield patches.Rectangle((s2+resolution/5, s1+resolution/5), width2, width1, **rectangle_kwargs)
        elif loc == "lower":
            yield patches.Rectangle((s1+resolution/5, s2+resolution/5), width1, width2, **rectangle_kwargs)
        else:
            raise ValueError("loc has to be uppper or lower")


# rep12 merged mcool r20k plot

## r20k Fig2A

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr1"
start = 16_500_000
end = 17_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_Fig2A_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_Fig2A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_Fig2A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Fig2B

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr9"
start = 13_790_000
end = 15_100_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_Fig2B_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_Fig2B_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_Fig2B_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()

## r20k Fig3A

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr8"
start = 88_500_000
end = 94_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_Fig3A_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_Fig3A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_Fig3A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Fig3C

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr8"
start = 119_000_000
end = 120_700_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_Fig3C_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_Fig3C_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_Fig3C_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Fig3E

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr1"
start = 9_800_000
end = 14_600_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_Fig3E_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_Fig3E_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_Fig3E_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Fig5A

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chrX"
start = 100_400_000
end = 103_400_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_Fig5A_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=2000, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_Fig5A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_Fig5A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


In [ ]:
# ### plot data in fall heatmap
# import matplotlib
# import matplotlib.pyplot as plt
# import cooltools.lib.plotting
# from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# # 设置为可编辑字体，默认的字体在AI中打开是路径
# matplotlib.rcParams['pdf.fonttype'] = 42
# # to plot ticks in terms of megabases we use the EngFormatter
# # https://matplotlib.org/gallery/api/engineering_formatter.html

# from matplotlib.ticker import EngFormatter
# bp_formatter = EngFormatter('b')
# def format_ticks(ax, x=True, y=True, rotate=True):
#     if y:
#         ax.yaxis.set_major_formatter(bp_formatter)
#     if x:
#         ax.xaxis.set_major_formatter(bp_formatter)
#         ax.xaxis.tick_bottom()
#     if rotate:
#         ax.tick_params(axis='x',rotation=45)

# chromsome = "chrX"
# start = 100_470_000
# end = 100_660_000
# plot_region = (chromsome, start, end)
# ax_limit_extents = (start, end, end, start)
# # Smoothing & Interpolation
# nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
#                                 nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
# nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
#                                 nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
# nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

# wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
#                                 wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
# wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# # interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
# figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
# figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
# figure.suptitle(f'Nora_Fig5A_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# # ax1
# ax1 = axes[0]
# im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=3000, cmap='fall', extent=ax_limit_extents)
# ax1.set_title('Nora_WT_r20k', y=1.01)
# ax1.set_ylabel('position(Mb)')
# # ax2
# ax2 = axes[1]
# im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=3000, cmap='fall', extent=ax_limit_extents)
# ax2.set_title('Nora_IAA_r20k', y=1.01)
# # ax3
# ax3 = axes[2]
# im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=3000, cmap='fall', extent=ax_limit_extents)
# ax3.set_title('WN_BPK25_r20k', y=1.01)

# for ax in axes:
#     format_ticks(ax)
#     ax.set_xlabel('position(Mb)')
# plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# plt.savefig("Nora_Fig5A_use_Nora_WT_IAA_WN_BPK25.zoomIn.rep12merged.r20k.fall.heatmap.pdf")
# plt.savefig("Nora_Fig5A_use_Nora_WT_IAA_WN_BPK25.zoomIn.rep12merged.r20k.fall.heatmap.png")
# plt.show()


## r20k FigS3I

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr3"
start = 39_900_000
end = 42_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_FigS3I_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_FigS3I_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_FigS3I_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k FigS3J

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr14"
start =109_400_000
end = 111_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=4, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nora_FigS3J_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("Nora_FigS3J_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("Nora_FigS3J_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Malat1 flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr19"
start = 5_000_000
end = 6_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Malat1_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Malat1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Malat1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Myc flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr15"
start = 61_000_000
end = 63_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Myc_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Myc_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Myc_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Nanog flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr6"
start = 121_500_000
end = 123_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Nanog_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Nanog_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Nanog_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Pou5f1 flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr17"
start = 34_500_000
end = 36_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Pou5f1_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Pou5f1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Pou5f1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Sox2 flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr3"
start = 33_500_000
end = 35_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Sox2_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Sox2_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Sox2_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Pax6 flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr2"
start = 104_500_000
end = 106_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Pax6_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Pax6_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Pax6_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Sox1 flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr8"
start = 11_000_000
end = 13_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Sox1_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1000, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1000, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1000, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Sox1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Sox1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Zic1 flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

chromsome = "chr9"
start = 90_000_000
end = 92_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Zic1_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Zic1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Zic1_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


In [ ]:
# 20251112 add  HoxA/B/C/D flank heat map  (new code add start)

## r20k Hoxa* flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

# hoxa* center chr6:52_200_000
chromsome = "chr6"
start = 51_100_000
end = 53_300_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Hoxa_cluster_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Hoxa_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Hoxa_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Hoxb* flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

# hoxb* center chr11:96_300_000
chromsome = "chr11"
start = 95_300_000
end = 97_300_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Hoxb_cluster_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1400, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1400, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1400, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Hoxb_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Hoxb_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Hoxc* flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

# hoxc* center chr15:103_000_000
chromsome = "chr15"
start = 102_200_000
end = 103_800_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Hoxc_cluster_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1800, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1600, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1700, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Hoxc_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Hoxc_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


## r20k Hoxd* flank

In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

# hoxd* center chr2:74_700_000
chromsome = "chr2"
start = 73_700_000
end = 75_700_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

nora_iaa_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
nora_iaa_rep12_mcool_r20k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r20k_adcg)

wn_bpk25_rep12_mcool_r20k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=9)
wn_bpk25_rep12_mcool_r20k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r20k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle(f'Hoxd_cluster_flank_region ({chromsome}:{start:,}-{end:,})', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r20k', y=1.01)
ax1.set_ylabel('position(Mb)')
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r20k', y=1.01)
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r20k', y=1.01)

for ax in axes:
    format_ticks(ax)
    ax.set_xlabel('position(Mb)')
plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)

# draw rectangular "boxes" around pixels called as tads in the "region":
for box in rectangles_around_tads(nora_wt_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax1.add_patch(box)
for box in rectangles_around_tads(nora_iaa_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax2.add_patch(box)
for box in rectangles_around_tads(wn_bpk25_rep12_r20k_tads_df, plot_region, lw=1.5, loc="none"):
    ax3.add_patch(box)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)
for box in rectangles_around_dots(nora_iaa_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax2.add_patch(box)
for box in rectangles_around_dots(wn_bpk25_rep12_r20k_loops_df, 20000, plot_region, lw=1.5, loc="upper"):
    ax3.add_patch(box)

plt.savefig("view_Hoxd_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf")
plt.savefig("view_Hoxd_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png")
plt.show()


In [ ]:
%%bash
mv view_Hoxa_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/
mv view_Hoxa_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/

mv view_Hoxb_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/
mv view_Hoxb_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/

mv view_Hoxc_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/
mv view_Hoxc_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/

mv view_Hoxd_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/
mv view_Hoxd_cluster_flank_use_Nora_WT_IAA_WN_BPK25.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/


In [ ]:
# 20251112 add  HoxA/B/C/D flank heat map  (new code add end)

In [ ]:
import os
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline/")
print(os.getcwd())

In [ ]:
%%bash
mkdir comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k
mv *Nora_WT_IAA_WN_BPK25*.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/
mv *Nora_WT_IAA_WN_BPK25*.rep12merged.r20k.fall.heatmap.with_tads_and_loops.pdf ./comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k/


In [ ]:
! find . -name *Nora_WT_IAA_WN_BPK25*.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png

In [ ]:
import os
import glob
import platform

from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

os_name = platform.system()
print(os_name)
# 指定目录路径
directory = './comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k'
# 构建文件路径的通配符
file_pattern = os.path.join(directory, '*.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png')
# 使用 glob 模块获取匹配的文件路径列表
png_files = sorted(glob.glob(file_pattern))
print(len(png_files))
# 打印文件路径列表
for file in png_files:
    # print(file)
    if os_name == 'Windows':
        print(file.split('\\')[2])
    else:
        print(file.split('/')[2])


# add rep12 merged r20k plot to summary ppt

In [ ]:
import os
import glob
import platform

from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

os_name = platform.system()
# 指定目录路径
directory = './comparable_heatmap_with_tads_and_loops_use_Nora_WT_IAA_WN_BPK25_r20k'
# 构建文件路径的通配符
file_pattern = os.path.join(directory, '*.rep12merged.r20k.fall.heatmap.with_tads_and_loops.png')
# 使用 glob 模块获取匹配的文件路径列表
png_files = sorted(glob.glob(file_pattern))
# print(len(png_files))

# 创建 PPT 并插入图片
my_ppt = Presentation()
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

page_count = len(png_files)
# print(page_count)
# jpg 图像文件处理序号
pi = 1
for i in range(1, page_count+1):
    # print(pi)
    slide = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
    # top
    if pi <= len(png_files):
        # 插入文本框
        # text_box = slide.shapes.add_textbox(left, top, width, height)
        text_box = slide.shapes.add_textbox(Cm(8.80), Cm(0), Cm(15.9), Cm(0.5))
        # 设置文本框背景颜色为黄色
        text_box.fill.solid()
        text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
        # 在文本框中添加内容并设置格式
        text_frame = text_box.text_frame
        if os_name == 'Linux':
            text_frame.text = png_files[pi-1].split('/')[2]
        elif os_name == 'Windows':
            text_frame.text = png_files[pi-1].split('\\')[2]
        # 设置字体大小和加粗
        text_font = text_frame.paragraphs[0].font
        text_font.name = '微软雅黑'
        text_font.size = Pt(6)
        text_font.bold = True
        # PPT 页面指定区域插入图片
        # pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
        pic = slide.shapes.add_picture(png_files[pi-1], Cm(8.80), Cm(0.50), Cm(15.9), Cm(6.6))
    # 页面索引 +1 
    pi += 1

# 保存 PPT
my_ppt.save('6_Nora_WT_IAA_WN_BPK25.rep12merged.fall.heatmap.with_tads_and_loops.summary.pptx')
